Осуществим предобработку данных с Твиттера, чтобы очищенные данные в дальнейшем
использовать для задачи классификации. Данный датасет содержит негативные (label = 1)
и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.
Задания:
1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим
функцию:
● для того, чтобы найти все вхождения паттерна в тексте, необходимо
использовать re.findall(pattern, input_txt)
● для для замены @user на пробел, необходимо использовать re.sub()
2. Изменим регистр твитов на нижний с помощью .lower().
3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя
apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в
тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в
качестве ключа (сокращенного слова), то заменить ключ на значение (полную
версию слова).
4. Заменим сокращения на их полные формы, используя short_word_dict. Для этого
воспользуемся функцией, используемой в предыдущем пункте.
5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict.
Для этого воспользуемся функцией, используемой в предыдущем пункте.
6. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.
7. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.
8. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.
9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if
len(w)>1]).
10. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый
столбец 'tweet_token'.
11. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец
'tweet_token_filtered' без стоп-слов.
12. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим
столбец 'tweet_stemmed' после применения стемминга.
13. Применим лемматизацию к токенам с помощью
nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после
применения лемматизации.
14. Сохраним результат предобработки в pickle-файл.

In [90]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [60]:
test = pd.read_csv('/content/sample_data/test_tweets.csv')
train = pd.read_csv('/content/sample_data/train_tweets.csv')

In [61]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [62]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [63]:
df = pd.concat([train, test])
df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [64]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49159 entries, 0 to 17196
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ MB
None


**1.** Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: ● для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt) ● для для замены @user на пробел, необходимо использовать re.sub()

In [69]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'@[\w]*', ' ', x))

In [71]:
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


**2.** Изменим регистр твитов на нижний с помощью .lower().

In [74]:
df['tweet'] = df['tweet'].str.lower()
df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...
6,7,0.0,camping tomorrow dannyâ¦
7,8,0.0,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,welcome here ! i'm it's so #gr8 !


**3.** Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [76]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [77]:
def change_by_dict(tweet, _dict):
    tweet_new = ''
    for word in tweet.split():
        word_new = _dict[word] if word in _dict else word
        tweet_new = tweet_new + word_new + " "
    return tweet_new

In [80]:
df['tweet'] = df['tweet'].apply(lambda x: change_by_dict(x, apostrophe_dict))
df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for #lyft credit i cannot use cause the...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ur...
4,5,0.0,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...
6,7,0.0,camping tomorrow dannyâ¦
7,8,0.0,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,welcome here ! I am it has / it is so #gr8 !


**4.** Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.


In [81]:
df['tweet'] = df['tweet'].apply(lambda x: change_by_dict(x, short_word_dict))
df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for #lyft credit i cannot use cause the...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love you take with you all the time i...
4,5,0.0,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...
6,7,0.0,camping tomorrow dannyâ¦
7,8,0.0,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,welcome here ! I am it has / it is so #gr8 !


**5.** Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.


In [82]:
df['tweet'] = df['tweet'].apply(lambda x: change_by_dict(x, emoticon_dict))
df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for #lyft credit i cannot use cause the...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love you take with you all the time i...
4,5,0.0,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...
6,7,0.0,camping tomorrow dannyâ¦
7,8,0.0,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,welcome here ! I am it has / it is so #gr8 !


**6.** Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

In [83]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for lyft credit i cannot use cause the...
2,3,0.0,bihday your majesty
3,4,0.0,model i love you take with you all the time i...
4,5,0.0,factsguide society now motivation
5,6,0.0,2 2 huge fan fare and big talking before the...
6,7,0.0,camping tomorrow dannyâ
7,8,0.0,the next school year is the year for exams ð ...
8,9,0.0,we won love the land allin cavs champ...
9,10,0.0,welcome here I am it has it is so gr8


**7.** Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

In [84]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))
df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for lyft credit i cannot use cause the...
2,3,0.0,bihday your majesty
3,4,0.0,model i love you take with you all the time i...
4,5,0.0,factsguide society now motivation
5,6,0.0,2 2 huge fan fare and big talking before the...
6,7,0.0,camping tomorrow danny
7,8,0.0,the next school year is the year for exams ...
8,9,0.0,we won love the land allin cavs champ...
9,10,0.0,welcome here I am it has it is so gr8


**8.** Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.


In [85]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for lyft credit i cannot use cause the...
2,3,0.0,bihday your majesty
3,4,0.0,model i love you take with you all the time i...
4,5,0.0,factsguide society now motivation
5,6,0.0,huge fan fare and big talking before the...
6,7,0.0,camping tomorrow danny
7,8,0.0,the next school year is the year for exams ...
8,9,0.0,we won love the land allin cavs champ...
9,10,0.0,welcome here I am it has it is so gr


**9.** Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if
len(w)>1]).


In [87]:
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
df.head(10)

,id,label,tweet
0,1,0.0,when father is dysfunctional and is so selfish...
1,2,0.0,thanks for lyft credit cannot use cause they d...
2,3,0.0,bihday your majesty
3,4,0.0,model love you take with you all the time in ur
4,5,0.0,factsguide society now motivation
5,6,0.0,huge fan fare and big talking before they leav...
6,7,0.0,camping tomorrow danny
7,8,0.0,the next school year is the year for exams can...
8,9,0.0,we won love the land allin cavs champions clev...
9,10,0.0,welcome here am it has it is so gr


**10.** Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый
столбец 'tweet_token'.

In [94]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [95]:
df['tweet_token'] = df['tweet'].apply(lambda x: nltk.tokenize.word_tokenize(x))
df.head(10)

,id,label,tweet,tweet_token
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ..."
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]"
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t..."
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]"
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ..."
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi..."
9,10,0.0,welcome here am it has it is so gr,"[welcome, here, am, it, has, it, is, so, gr]"


**11.** Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец
'tweet_token_filtered' без стоп-слов.


In [100]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [105]:
stop_words = stopwords.words('english')

df['tweet_token_filtered'] = df['tweet_token'].apply(lambda text: [word for word in text if not word in stop_words])
df.head(10)

,id,label,tweet,tweet_token,tweet_token_filtered
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]"
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t...","[huge, fan, fare, big, talking, leave, chaos, ..."
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]","[camping, tomorrow, danny]"
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ...","[next, school, year, year, exams, think, schoo..."
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi...","[love, land, allin, cavs, champions, cleveland..."
9,10,0.0,welcome here am it has it is so gr,"[welcome, here, am, it, has, it, is, so, gr]","[welcome, gr]"


**12.** Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим
столбец 'tweet_stemmed' после применения стемминга.


In [107]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [108]:
df['tweet_stemmed'] = df['tweet_token_filtered'].apply(lambda text: [stemmer.stem(word) for word in text])

df.head(10)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]"
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t...","[huge, fan, fare, big, talking, leave, chaos, ...","[huge, fan, fare, big, talk, leav, chao, pay, ..."
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]","[camping, tomorrow, danny]","[camp, tomorrow, danni]"
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ...","[next, school, year, year, exams, think, schoo...","[next, school, year, year, exam, think, school..."
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi...","[love, land, allin, cavs, champions, cleveland...","[love, land, allin, cav, champion, cleveland, ..."
9,10,0.0,welcome here am it has it is so gr,"[welcome, here, am, it, has, it, is, so, gr]","[welcome, gr]","[welcom, gr]"


**13.** Применим лемматизацию к токенам с помощью
nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после
применения лемматизации.


In [109]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [110]:
df['tweet_lemmatized'] = df['tweet_token_filtered'].apply(lambda text: [lemmatizer.lemmatize(word) for word in text])

df.head(10)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t...","[huge, fan, fare, big, talking, leave, chaos, ...","[huge, fan, fare, big, talk, leav, chao, pay, ...","[huge, fan, fare, big, talking, leave, chaos, ..."
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]","[camping, tomorrow, danny]","[camp, tomorrow, danni]","[camping, tomorrow, danny]"
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ...","[next, school, year, year, exams, think, schoo...","[next, school, year, year, exam, think, school...","[next, school, year, year, exam, think, school..."
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi...","[love, land, allin, cavs, champions, cleveland...","[love, land, allin, cav, champion, cleveland, ...","[love, land, allin, cavs, champion, cleveland,..."
9,10,0.0,welcome here am it has it is so gr,"[welcome, here, am, it, has, it, is, so, gr]","[welcome, gr]","[welcom, gr]","[welcome, gr]"


**14.** Сохраним результат предобработки в pickle-файл.

In [111]:
df.to_pickle('/content/sample_data/df_tweets.pkl')